### Tipo de grafo selecionado: 
### Cálculo do Grau de Centralidade (Degree Centrality)
### Cálculo Centralidade do Autovetor / Valor (Eigenvector)
### Betweenness
### Closeness

In [1]:
import networkx as nx
import ibm_db
import ibm_db_dbi
import pandas as pd
from pyvis.network import Network
from itertools import combinations
import time

### Função cria_grafo

In [6]:
def cria_grafo(user,pwd):
    
    db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID="+user+";PWD="+pwd+";", "", "")
    conn = ibm_db_dbi.Connection(db) 
    
    # Selecionando ocorrência e pessoas de interesse
    query = """ SELECT
                    oc.NRO_INT_OCORRENCIA,
                    poc.NRO_INT_PESSOA_INTERESSE, 
                    tp.TXT_DESCRICAO VINCULO
                FROM
                    GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                    INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                    INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                    INNER JOIN GESEG_TARGET.GESEG_DIM_TIPO_PARTICIPACAO tp ON poc.NRO_INT_TIPO_PARTICIPACAO = tp.NRO_INT_TIPO_PARTICIPACAO -- Coloquei para descobrir participação pessoa de interesse
                WHERE
                    ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                    AND YEAR(DATA_FATO) IN (2016,2017)
                    AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                GROUP BY oc.NRO_INT_OCORRENCIA, poc.NRO_INT_PESSOA_INTERESSE,tp.TXT_DESCRICAO;"""
    
    df = pd.read_sql_query(query, conn)
    # title é utilizado para aparecer o tipo de vínculo quando o mouse passar pela aresta
    df['title'] = df['VINCULO']
    
    #informando quais colunas da tabela serão o source, target e demais atributos;
    D = nx.from_pandas_edgelist(df, source = 'NRO_INT_OCORRENCIA', target = 'NRO_INT_PESSOA_INTERESSE', edge_attr= True, create_using=nx.Graph())
    
    # definindo o atributo 'tipo_no' para todos os nós
    for n in D.nodes:
        D.nodes[n].setdefault('TIPO_NO', 'sem_descricao')
        
    # NÓ OCORRÊNCIA
    #buscando os atributos do vértice 
    query = """SELECT
                    oc.NRO_INT_OCORRENCIA,
                    ft.DESC_FATO Fato,
                    DATA_REGISTRO,
                    HORA_REGISTRO,
                    DATA_FATO,
                    HORA_FATO,
                    count(*) envolvimentos,
                    count(DISTINCT poc.NRO_INT_PESSOA_INTERESSE) pessoas_envolvidas
                FROM
                    GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                    INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                    INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                WHERE
                    ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                    AND YEAR(DATA_FATO) IN (2016,2017)
                    AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                GROUP BY oc.NRO_INT_OCORRENCIA, DATA_REGISTRO, DATA_FATO, ft.DESC_FATO, HORA_FATO, HORA_REGISTRO;"""

    #TRANSFERINDO DADOS PARA TABELA
    df_node_ocor = pd.read_sql_query(query, conn)
    
    #Criando shape e cor diferente para os nós ocorrência
    df_node_ocor['TIPO_NO'] = 'ocorrencia'
    df_node_ocor['NUMERO'] = df_node_ocor['NRO_INT_OCORRENCIA']
    df_node_ocor['shape'] = 'triangle'
    df_node_ocor['color'] = 'orange'
    df_node_ocor['label'] = ' '
    df_node_ocor['title'] = 'Ocorrência: ' + df_node_ocor['NRO_INT_OCORRENCIA'].astype(str) + '\n Fato: ' + df_node_ocor['FATO']

    #formatando o atributo Data
    df_node_ocor['DATA_FATO'] = pd.to_datetime(df_node_ocor['DATA_FATO'])
    df_node_ocor['DATA_FATO'] = df_node_ocor['DATA_FATO'].dt.strftime('%Y-%m-%d')
    df_node_ocor['DATA_REGISTRO'] = pd.to_datetime(df_node_ocor['DATA_REGISTRO'])
    df_node_ocor['DATA_REGISTRO'] = df_node_ocor['DATA_REGISTRO'].dt.strftime('%Y-%m-%d')
    
    #transformando a tabela em um dicionário para atribuir os dados aos vértices 'ocorrencia'
    node_ocorr_atrib = df_node_ocor.set_index("NRO_INT_OCORRENCIA").T.to_dict()
    # atributo 
    nx.set_node_attributes(D, node_ocorr_atrib)
    
    #NÓ PESSOA DE INTERESSE
    # atributos do nó
    query = """SELECT
                    DISTINCT pi.NRO_INT_PESSOA_INTERESSE,
                    pi.NOME
                FROM
                    GESEG_TARGET.GESEG_VER_PESSOA_INTERESSE pi
                WHERE
                    NRO_INT_PESSOA_INTERESSE IN (
                    SELECT 
                        poc.NRO_INT_PESSOA_INTERESSE
                    FROM
                        GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                        INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                        INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                    WHERE ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                        AND YEAR(DATA_FATO) IN (2016,2017)
                        AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                    );"""
    
    #TRANSFERINDO DADOS PARA TABELA
    df_node_pi = pd.read_sql_query(query, conn)
    
    #Retirando o label dos nós
    df_node_pi['TIPO_NO'] = 'pessoa'
    df_node_pi['label'] = ' '
    df_node_pi['title'] = df_node_pi['NOME']
    df_node_pi['NUMERO'] = df_node_pi['NRO_INT_PESSOA_INTERESSE']
    
    #transformando a tabela em um dicionário para atribuir os dados aos vértices 'pesso interesse'
    node_ocorr_atrib2 = df_node_pi.set_index("NRO_INT_PESSOA_INTERESSE").T.to_dict()
    # definindo atributos
    nx.set_node_attributes(D, node_ocorr_atrib2)
    
    ### Co- Autores
    #Criando vínculo co-autores
    for n in D.nodes:
        if D.nodes[n]['TIPO_NO'] == 'ocorrencia':
            lista_vizinhos = D.neighbors(n)
            liga_vizinhos = combinations(lista_vizinhos, 2) 
            D.add_edges_from(liga_vizinhos, VINCULO ='co-autores')
            
    # Verificando quais nós possuem mais de uma ligação 'co-autor' entre eles
    #duplos = [(u, v, k, d) for u, v, k, d in D.edges(data=True, keys = True) if d['VINCULO'] == 'co-autores' and k != 0]

    # formatando os edges que desejamos excluir
    #duplos_format = []
    #for i in range(len(duplos)):
        #duplos_format.append(duplos[i][:3])
        
    # removendo ligações de co-autores duplicados
    #D.remove_edges_from(duplos_format)
    
    # colocando o atributo 'title' para aparecer qual é a relação quando o mouse parar em cima da aresta
    for u,v,d in D.edges(data=True):
        if d['VINCULO'] == 'co-autores':
            D[u][v]['title'] = 'co-autores'
            
    conn.close()
    return D;

In [3]:
# plotar o grafo
def plot_g_pyviz(G, name='out.html', height='300px', width='100%',notebook=True, directed=False):
    g = G.copy() # some attributes added to nodes
    net = Network(notebook=notebook, directed=directed, height=height, width=width)
    opts = '''
        var options = {
          "physics": {
            "forceAtlas2Based": {
              "gravitationalConstant": -100,
              "centralGravity": 0.11,
              "springLength": 100,
              "springConstant": 0.09,
              "avoidOverlap": 1
            },
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based",
            "timestep": 0.22
          }
        }
    '''

    net.set_options(opts)
    # uncomment this to play with layout
    # net.show_buttons(filter_=['physics'])
    net.from_nx(g)
    return net.show(name)

### Criando grafo

In [7]:
# CRIANDO GRAFO

#COLOCAR CREDENCIAIS para acessar o banco
user = "milena-villar"
pwd = "inicial@19"

t1 = time.time()
# chamando função
G = cria_grafo(user, pwd)
tempoExec = time.time() - t1
print("Tempo de execução criação de grafo: {} segundos".format(tempoExec))

Tempo de execução criação de grafo: 12.857410669326782 segundos


In [8]:
# verificando se todos os nós estão com o 'tipo' definido
teste = (n for n in G if G.nodes[n]['TIPO_NO']== 'sem_descricao')
print(sorted(teste))

['4025CAD23B7ED014F4A421D5866337C7']


In [9]:
# Verificando se os edges 'co-autores' duplicados foram excluídos
G['1495C1291F438A12CBE7284F2383F2F1']['71B94064FD47CA2D303503957F0205F2']

{'VINCULO': 'co-autores', 'title': 'co-autores'}

In [24]:
# Testando co-autores // ,32309971
T2= nx.generators.ego_graph(G, 31025120 , radius=1, center = True, undirected = True)

# plotar o grafo
plot_g_pyviz(T2,name='multigrafo2016.html' ,notebook = True, directed = False)

In [15]:
print('número de vértices: ',G.number_of_nodes())
print('número de arestas: ',G.number_of_edges())

número de vértices:  18367
número de arestas:  20953


In [16]:
# Comunidades

import community

partition = community.best_partition(G)
values = [partition.get(node) for node in G.nodes()]
print("Número de comunidades encontradas: ", len(set(values)))

Número de comunidades encontradas:  6349


In [17]:
import numpy as np

deg = nx.degree(G)
deg_seq = [d for n,d in deg]

nos = G.number_of_nodes()
grau_med = sum(deg_seq)/ nos

print("Grau médio dos nós: %.2f" % grau_med)

Grau médio dos nós: 2.28


 O critério de Molloy-Reed busca demonstrar a existência, ou ausência, de um componente gigante em uma rede aleatória.
Seguindo o principio que para um componente gigante existir em uma rede, a maioria dos nós precisam estar conectados a outros dois nós, pelo menos.

In [18]:
# Componente Gigante ( maior cluster conectado da rede)
largest_cc = max(nx.connected_components(G), key=len)
G0 = G.subgraph(largest_cc)
# plotar o grafo
plot_g_pyviz(G0, name='cg2016_2017.html', height='100%',notebook = False, directed = False)

Conforme o artigo: "nos concentramos apenas no componente gigante da rede, uma vez que o maior componente conectado pode representar uma fase do crime generalizado e auto-organizado, mais perigoso do ponto de vista da segurança nacional."

In [19]:
pver = (G0.number_of_nodes()/G.number_of_nodes())*100
aver = (G0.number_of_edges()/G.number_of_edges())*100
print('número de vértices: '+ str(G0.number_of_nodes()) + '  {:01.0f}% do número total de nós'.format(pver))
print('número de arestas: ' + str(G0.number_of_edges()) + '  {:02.0f}% do número total de nós'.format(aver))

número de vértices: 967  5% do número total de nós
número de arestas: 3165  15% do número total de nós


In [20]:
deg0 = nx.degree(G0)
deg_seq0 = [d for n,d in deg0]

nos0 = G0.number_of_nodes()
grau_med0 = sum(deg_seq0)/ nos0

print("Grau médio dos nós: {:01.2f}".format(grau_med0))

Grau médio dos nós: 6.55


In [21]:
# Comunidades G0
partition_G0 = community.best_partition(G0)
values0 = [partition_G0.get(node) for node in G0.nodes()]
print("Número de comunidades encontradas: ", len(set(values0)))

Número de comunidades encontradas:  22


In [22]:
# Configurando as cores das comunidades e plotando o grafo
color_list = ["#c357c5","#52c35e","#785cce","#87bb37","#507ddb","#c2b73d","#b78ddf","#4e8f2d","#d6438e","#61c38e","#d8425b",
              "#45c7c8","#db542e","#629fd6","#dc9030","#6561a3","#938a2a","#9a4c87","#438a4d","#dd86b6","#2a6a45","#a84927",
              "#38987e","#a54a5b","#9eb46b","#e48374","#616f2a","#d3a166","#90652d"]

color_list = color_list[0:len(set(values0))]
color_dict = pd.Series(color_list, index=np.arange(0,len(set(values0)))).to_dict()

for key, value in partition_G0.items():
    partition_G0[key] = color_dict[value]

nx.set_node_attributes(G0, partition_G0, 'color')
plot_g_pyviz(G0, name='cg2016_2017_comunidades.html', height='100%', notebook = False, directed = False)

Conforme o artigo: " A densidade e a eficiência da rede nos informam sobre o compromisso entre a segurança e a difusão efetiva de informações e dados"

A baixa densidade e eficiência estão relacionados à clandestinidade da rede que tenta constantemente se esconder da vigilância policial.

In [23]:
# Densidade: o quão esparsa é a rede; nível de conectividade
# Conforme o artigo: "em uma rede "brilhante" com um grande número de conexões entre criminosos, 
# a investigação ou captura de um ator ajudaria as autoridades a extrair informações críticas sobre a estrutura da rede.
# Uma rede mais escura, no entanto, embora oculte a estrutura das investigações, retardaria a transferência de informações
# dentro da rede devido aos caminhos mais longos entre os criminosos."
dens0 = nx.density(G0)
print("Densidade do grafo: {:04.4f}".format(dens0))

Densidade do grafo: 0.0068


In [41]:
# Eficiência da rede: se as conexões estão contribuindo para a redução do tamanho dos caminhos da rede
# Conforme o artigo: "a eficiência da rede quantifica a troca de informações"
efic0 = nx.global_efficiency(G0)
print("Eficiência do grafo: {:04.4f}".format(efic0))

Eficiência do grafo: 0.1970


In [27]:
# Coeficiente de aglomeração: coesão do grupo a que o nó pertence. Quanto mais os vizinhos do nó se conhecerem,
#maior é o coeficiente de aglomeração, mais coeso é o grupo.
md_ca = nx.average_clustering(G0)
md_cc = nx.average_shortest_path_length(G0)
print("Coeficiente de aglomeração médio: {:04.2f}".format(md_ca))
print("Comprimento médio de caminho mais curto: {:04.2f}".format(md_cc))

Coeficiente de aglomeração médio: 0.76
Comprimento médio de caminho mais curto: 6.45


A rede também é altamente hierárquica, refletindo o fato de que alguns indivíduos proeminentes são responsáveis pela coesão da rede, enquanto a maioria dos criminosos debaixo k participam apenas de uma pequena quantidade de interações.

## Algoritmos de Centralidade

### Grau de centralidade 
É uma medida do número de links diretos para outros nós na rede. É uma medida local, pois seu valor é calculado com base no número de links que um nó tem com os outros nós diretamente adjacentes a ele. Atores em redes sociais com um alto grau de centralidade servem como hubs e como principais canais de informação. Nas redes sociais, por exemplo, a distribuição de graus de nós segue uma distribuição de leis de potência, o que significa que muito poucos nós têm um número extremamente grande de conexões. Naturalmente, esses nós de alto grau têm mais impacto na rede do que outros nós e, portanto, são considerados mais importantes.

In [31]:
# grau de centralidade
t1 = time.time()
dg = nx.degree_centrality(G0)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')


Tempo de execução: 0.013 segundos
 


In [33]:
# Tabela
d = dict(G0.nodes(data="title", default=1))
id_nome = pd.DataFrame(list(d.items()),columns=['Num_Pessoa_Interesse', 'title'])
id_nome.set_index('Num_Pessoa_Interesse',inplace=True)

aux_dg = pd.DataFrame(list(dg.items()), columns=['Num_Pessoa_Interesse', 'Grau de Centralidade'])
aux_dg.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_dg = pd.concat([id_nome, aux_dg], axis=1)
rankings_dg.columns = ['Nome','Grau de Centralidade'] 
rankings_dg.sort_values('Grau de Centralidade', ascending = False, inplace = True)

N = 967
rankings_dg['Posição'] = range(1, N + 1)

rankings_dg.head(10).round(4)

,Nome,Grau de Centralidade,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.1480,1
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.0704,2
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.0663,3
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.0569,4
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.0538,5
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.0518,6
F0D4A6433AF58E5855BA79DD43BDD832,ERICK VINICIUS DA SILVA MATTOS,0.0455,7
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.0455,8
C348D60507F5D21F955EE964FCCCEBA0,DOUGLAS LOHAN RODRIGUES DA SILVA,0.0424,9


### Algoritmo Eigenvector (Centralidade do autovetor)
A medida de Centralidade do autovetor descreve a centralidade de um nó em relação à estrutura global da rede. Ela atribui pontuações relativas a todos os nós na rede com base no conceito de que as conexões para nós com alta pontuação contribuem mais do que as conexões para nós com baixa pontuação.
Ela mede a extensão em que um nó é conectado a nós bem conectados.
Nós com alta pontuação são considerados influentes por estarem conectados a nós importantes. Enquanto permanecem em grande parte nas sombras, eles podem explorar o poder dado por conhecer nós bem conectados.

In [34]:
t1 = time.time()
# centralidade do autovetor
ec = nx.eigenvector_centrality_numpy(G0) # função mais rápida; não precisa converter
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')

Tempo de execução: 0.348 segundos
 


In [36]:
# Tabela
aux_ec = pd.DataFrame(list(ec.items()), columns=['Num_Pessoa_Interesse', 'Eigenvector'])
aux_ec.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_ec = pd.concat([id_nome, aux_ec], axis=1)
rankings_ec.columns = ['Nome','Eigenvector'] 
rankings_ec.sort_values('Eigenvector', ascending = False, inplace = True)

N = 967
rankings_ec['Posição'] = range(1, N + 1)
rankings_ec.head(10).round(4)

,Nome,Eigenvector,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.3450,1
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.2585,2
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.2421,3
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.2348,4
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.2155,5
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.1965,6
6E7C0DA4D145EE7CE950C0B16F2A2DB6,WILLIAM OTAVIO OLIVEIRA DA SILVA,0.1871,7
C348D60507F5D21F955EE964FCCCEBA0,DOUGLAS LOHAN RODRIGUES DA SILVA,0.1799,8
05E01F13B8497FA74AE3241F37741A9B,PAULO RICARDO DA ROSA SANTOS,0.1714,9


### Algoritmo Betweenness (Centralidade de Interdependência)
Mede o número de caminhos mais curtos em que o nó se encontra. A Centralidade de Interdependência pode ser descrita como quão
importante um nó é, como um link entre diferentes redes. Nós com alta centralidade de interdependência controlam o fluxo de informação porque eles formam pontes críticas entre outros nós ou grupos de nós.
A Centralidade de Interdependência mostra quais nós são prováveis caminhos de informação e pode ser usado para determinar como um grafo irá separar os nós. Da mesma forma, é uma maneira de identificar aqueles que atuam como pontes (também chamadas de chaves de limite) entre dois ou mais segmentos de grafos que, de outra forma, não seriam capazes de se comunicar
entre si. Essa medida toca na importância que um ator pode ganhar estando no meio das comunicações sociais de uma rede e em que medida, em uma sociedade, ele é necessário como elo na cadeia de contatos.

In [116]:
t1 = time.time()
bc = nx.betweenness_centrality(G0)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')


Tempo de execução: 12.734 segundos
 


In [117]:
# Tabela
aux_bc = pd.DataFrame(list(bc.items()), columns=['Num_Pessoa_Interesse', 'Betweenness'])
aux_bc.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_bc = pd.concat([id_nome, aux_bc], axis=1)
rankings_bc.columns = ['Nome','Betweenness'] 
rankings_bc.sort_values('Betweenness', ascending = False, inplace = True)

N = 967
rankings_bc['Posição'] = range(1, N + 1)
rankings_bc.head(10).round(4)

,Nome,Betweenness,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.4955,1
0765A653C03AC699E2322F5280FE2CBD,WAGNER REIS DA SILVA,0.4033,2
B6C15C650538E504594A9CC68070241B,MAXIMILIANO DA SILVA PACHECO,0.3486,3
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.3298,4
938287B54D1F92232CB490C5890533E3,MARCOS DA SILVA OLIVEIRA,0.2412,5
F55E3058C48FB0ACD18812B5B230578F,ROMAR GONCALVES GOMES,0.2322,6
1D17711922B088BB40A63B849474B6C4,BRUNO FERNANDO SANHUDO TEIXEIRA,0.2039,7
81925BF90874257538F1C626C6F362D3,EDINILSON SILVA DOS SANTOS,0.1636,8
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.1219,9


### Algoritmo Closeness (Centralidade de Proximidade)
Esta medida de centralidade tenta expressar a importância de um vértice pelo fato dele estar próximo a muitos outros vértices. Esse vértice pode ser importante se estiver relativamente próximo do conjunto restante dos demais vértices na rede. A centralidade de proximidade é importante para entender a disseminação de informações em redes, de modo que a distância entre um nó em particular e outros tenha um efeito sobre como esse nó pode receber ou enviar informações para outros nós. Nas redes de informação, a proximidade revela quanto tempo leva para que um pouco de informação flua de um nó para outro na rede. Nós de alta pontuação geralmente têm caminhos mais curtos para o restante dos nós na rede.

In [118]:
t1 = time.time()
cc = nx.closeness_centrality(G0)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')

Tempo de execução: 18.752 segundos
 


In [119]:
# Tabela
aux_cc = pd.DataFrame(list(cc.items()), columns=['Num_Pessoa_Interesse', 'Closeness'])
aux_cc.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_cc = pd.concat([id_nome, aux_cc], axis=1)
rankings_cc.columns = ['Nome','Closeness'] 
rankings_cc.sort_values('Closeness', ascending = False, inplace = True)

N = 967
rankings_cc['Posição'] = range(1, N + 1)
rankings_cc.head(10).round(4)

,Nome,Closeness,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.2716,1
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.2481,2
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.2476,3
0765A653C03AC699E2322F5280FE2CBD,WAGNER REIS DA SILVA,0.2441,4
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.2420,5
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.2400,6
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.2388,7
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.2386,8
AC6D8A4288508D5ED1DD7D37AB133CC0,MARCELO ASSIS DOS SANTOS,0.2376,9


In [ ]:
#V.clear()